In [1]:
import requests
from requests.exceptions import RequestException
import re
import time

In [2]:
def get_one_page(url):  
    """
    爬取某一页的源代码；
    url：当前网页的url；
    return：当前网页的html文本
    """
    try:
        # 输入你的代码，构造请求方法，获取服务器响应 r；
        r = requests.get(url)
        if r.status_code == 200: 
            r.text.encode('utf-8')
            return r.text
        return None
    except RequestException as e:
        return None

def parse_one_page(html):
    """
    从当前网页中提取“排行榜号（list_num ）”，“书名”，“评论数量”，“推荐度”，
    “作者”、“出版社信息”、“原始价格”、“折后价格”，并存入字典中。
    """
    # 创建正则表达式对象，填充compile的第一个参数内
    pattern = re.compile(
        '<li>.*?list_num.*?(\d+).</div>.*?class="name".*?title="(.*?)">.*?class="star".*?target="_blank">(.*?)条评论</a>.*?class="tuijian">(.*?)%推荐<.*?class="publisher_info">.*?target="_blank">(.*?)</a>.*?class="publisher_info">.*?target="_blank">(.*?)</a>.*?class="price_n">&yen;(.*?)</span>.*?class="price_r">&yen;(.*?)</span>.*?</li>', re.S)
    
    items = re.findall(pattern, html)
    for item in items:
        yield {
            'index': item[0],  # 排行榜号-你的表达式 ,
            'title': item[1],  # 书名-你的表达式,
            'reviews': item[2],  # 评论数量-你的表达式,
            'recommends': item[3],  # 推荐度-你的表达式,
            'authors': item[4],  # 作者名-你的表达式,
            'publisher': item[5],  # 出版社-你的表达式
            'price_n': item[6],  # 折后价格-你的表达式,
            'price_r': item[7]  # 折前价格-你的表达式，
        }
        
        
def main(offset=1, file=open(r'data\dangdang_books.csv','w',encoding='utf-8')):
    """主函数
    offset：url的偏移量（从1到25）
    
    """
    url = 'http://bang.dangdang.com/books/bestsellers/01.00.00.00.00.00-24hours-0-0-1-' + str(offset)
    html = get_one_page(url)
    for item in parse_one_page(html):       
        for value in item.values():
            file.write('"'+str(value)+'"'+',')
        file.write('\n')
    file.close()

In [3]:
file = open(r'data\dangdang_books.csv','w',encoding='utf-8')
file.write('index,title,reviews,recommends(%),authors,publisher,price_n,price_r\n')
for i in range(1,26):
    url = 'http://bang.dangdang.com/books/bestsellers/01.00.00.00.00.00-24hours-0-0-1-' + str(i)
    html = get_one_page(url)
    for item in parse_one_page(html):
        line = ''
        for value in item.values():
            line += '"'+str(value)+'"'+','
        file.write(line.strip(',')+'\n')
    time.sleep(1)
file.close()

In [4]:
import pandas as pd

pd.read_csv(r'data\dangdang_books.csv',index_col=False)

,index,title,reviews,recommends(%),authors,publisher,price_n,price_r
0,1,你也走了很远的路吧（新增2万余字，4篇文章，关于特殊时期成长的勇气，以及曾经的那些人）,405117,100.0,卢思浩,湖南文艺出版社,24.9,49.80
1,2,蛤蟆先生去看心理医生（热销300万册！英国经典心理咨询入门书，知名心理学家李松蔚强烈推荐）,1508102,100.0,罗伯特・戴博德,天津人民出版社,19.0,38.00
2,3,青铜葵花（当当定制版）,318486,99.3,曹文轩,天天出版社有限责任公司,9.9,25.00
3,4,一读就上瘾的中国史1+2(套装全2册),72524,100.0,温伯陵,台海出版社,57.4,119.60
4,5,被讨厌的勇气：“自我启发之父”阿德勒的哲学课 岸见一郎,1027035,100.0,岸��一郎,机械工业出版社,33.0,55.00
...,...,...,...,...,...,...,...,...
495,496,当下的力量（珍藏版）,242950,99.9,埃克哈特.托利,中信出版社,36.4,56.00
496,497,奥特曼系列 视觉志:55周年纪念,36827,100.0,上海新创华文化发展有限公司,安徽少年儿童出版社,49.0,70.00
497,498,中国传统节日故事自营儿童注音版（美绘大开本）一二三年级儿童文学 小学生课外阅读 全国著名语文...,42189,100.0,金波,北京教育出版社,23.2,25.80
498,499,吹牛大王历险记（彩图注音版） 汇聚金波 沈石溪 常新港 冰波 汤素兰等50多位名家名作,81926,100.0,拉斯伯,北京教育出版社,12.9,19.80
